In [2]:
import requests as req
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
import string
#from tqdm import tqdm
import time

In [3]:
headers_s = {
'authority': 'www.goodrx.com',
'method': 'GET',
'path': '/optimizely-edge/18262057259?url=https%3A%2F%2Fwww.goodrx.com%2Fdrugs%2Fa',
'scheme': 'https',
'accept': '*/*',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en-US,en;q=0.9',
'cookie': 'myrx_exp_ab_variant=experiment; myrx_exp_ab_variant=experiment; grx_unique_id=2a004c803e02427bac4eac3ce9d9c293; optimizelyEndUserId=2a004c803e02427bac4eac3ce9d9c293; isVisitedHomepage=true; grx_visit_start=1648147208; grx_sa=false; gclid=CjwKCAjwrfCRBhAXEiwAnkmKmf49NS0XrI8X4ILkuTO5GTmtMYZf6R_QO7ONQa4s-QzVGN8nhZ_6WBoCTEkQAvD_BwE; osano_consentmanager_uuid=0bb4abd9-968b-4cae-a8ca-22c6e4d1779a; osano_consentmanager=4V0Y0gJFLzPttfaFscNcaewcCwYqleVNJmMd-AsmEmu7CjzbeNNvGY1vPC_VGQ17-0SFfuFXoMS-1LJ9ybZejTneZQsRI2nDmAGoLADbiKXSXwO3eljMc42NTmM_vnMgTIA_3HMVU1PvdlkQsmzWh-ht5WxqI65jx4bG-tY8-TYxBw127lnEIMxWFLsbAcFc0QyEgLyThBYAlD_q8GmO24Mu_J5ahCIVdHzrfPNW69DAi7uzkxMP3Cc4F5PVRt0vQegAhCVxw33rLt5S60F43iIuttT__mkgyFNFAA==; __gtm_campaign_url=https://www.goodrx.com/?utm_campaign=11601593411&utm_content=114733542162&utm_source=google&utm_medium=cpc&utm_term=kwd-304031349979&gclsrc=aw.ds&gclid=CjwKCAjwrfCRBhAXEiwAnkmKmf49NS0XrI8X4ILkuTO5GTmtMYZf6R_QO7ONQa4s-QzVGN8nhZ_6WBoCTEkQAvD_BwE; grx_cm_signal=1; ajs_anonymous_id="2a004c803e02427bac4eac3ce9d9c293"; gclid=CjwKCAjwrfCRBhAXEiwAnkmKmf49NS0XrI8X4ILkuTO5GTmtMYZf6R_QO7ONQa4s-QzVGN8nhZ_6WBoCTEkQAvD_BwE; pxcts=d6b947d7-aba1-11ec-b127-7a7054484365; _pxvid=d4501d79-aba1-11ec-af63-4a6f4a4d777a; _ga=GA1.2.427663771.1648147210; _gid=GA1.2.1296306263.1648147210; gdna_rid=XY2280WKxNqiqVHuqKpOyZApFXyaU3zKaipTOD-J-EG_FyJWg; __psrw=3693aacc-0236-4c80-b7fa-59eafdc858c3; __psugroup=2; _gac_UA-24914838-1=1.1648147237.CjwKCAjwrfCRBhAXEiwAnkmKmf49NS0XrI8X4ILkuTO5GTmtMYZf6R_QO7ONQa4s-QzVGN8nhZ_6WBoCTEkQAvD_BwE; csrf_token=dc1c10c205914bbf9ecc78978f37e652; client=; external_attribution_id=; c=; kw=; grx_experiment_variation=adaptive; grx_experiment_id=price_page_refresh_1_5; ppa_exp_ab_variant=experiment; _parsely_visitor={"id":"pid=a52bac94618cee2e32d05aaf5408ec93","session_count":1,"last_session_ts":1648147259553}; __gads=ID=f00bf3152ddfc653:T=1648147261:S=ALNI_MYqTX2U3EFUnXXPbWVohdLWrijGXw; QSI_SI_4YE6UZfiQYe05DM_intercept=true; ki_u=05972acc-0e17-ae0a-1e3a-0b21; ki_s=213227:2.0.0.0.2; __gtm_referrer=https://www.google.com/; grx_experiment_data=eyJhdHRyaWJ1dGVzIjpbeyJlbnRpdHlfaWQiOiIxODkyMDY1Mzg3NyIsImtleSI6ImVudmlyb25tZW50IiwidHlwZSI6ImN1c3RvbSIsInZhbHVlIjoicHJvZCJ9XSwiZGVjaXNpb25zIjp7ImNhbXBhaWduX2lkIjoiMjEyMzMzNDMwMjgiLCJleHBlcmltZW50X2lkIjoiMjEyMTg5MTA2NjUiLCJ2YXJpYXRpb25faWQiOiIyMTI0MDc3MTkwNSJ9LCJleHBlcmltZW50S2V5IjoicHJpY2VfcGFnZV9yZWZyZXNoXzFfNSIsIm9yaWdpbmFsVXJsIjoiL2lzb21ldGhlcHRlbmUtZGljaGxvcmFscGhlbmF6b25lLWFjZXRhbWlub3BoZW4iLCJ2YXJpYXRpb24iOiJhZGFwdGl2ZSJ9; grx.hcp.dgid=Not a provider; ki_r=; _pxhd=N5EzrdpB6rA7hm27HJmUbPynioW/so7oYTDESD-af-PZkK8E2mwFicFo/XhbsP9gbV9-xWVyfsoemgVW1qvUsg==:-x0zOljIF2-SYntT3h0CW6snlnaTzgxPUc7/D9fQCpzRsIEv1-S8JdR0wM/GLTz48S6Xpv0SscTRMEVNqeVq1ByAGEJt1tImY1zEtj8eBho=; lux_uid=164825498384964607; landingCategory=34; _sess=13774391-37ff-4e92-a7be-01119813a597.a37e7ba2-9d8f-45f9-8664-ac8ed089c514.1648254986.1; dmd-vid=a37e7ba2-9d8f-45f9-8664-ac8ed089c514; dmd-sid=13774391-37ff-4e92-a7be-01119813a597; dmd-ahk=43b30bea05; dmd-signal-197-3055-0B188DA5-13774391-37ff-4e92-a7be-01119813a597=e30=; recentSearchesCWF=[{"display":"cetirizine / pseudoephedrine (Generic Zyrtec-D)","slug":"zyrtec-d","type":"GENERIC","url":"/zyrtec-d"},{"display":"isometheptene / dichloralphenazone / acetaminophen (Generic Epidrine Midrin)","slug":"isometheptene-dichloralphenazone-acetaminophen","type":"GENERIC","url":"/isometheptene-dichloralphenazone-acetaminophen"}]; dmd-signal-false-13774391-37ff-4e92-a7be-01119813a597=e30=; grx_location={"location":{"state":"KS","postalCode":"66070","city":"ozawkie","country":"US"}}; _px2=eyJ1IjoiMTNhNTdjODAtYWNhMC0xMWVjLThkMzgtZmQ3NzgzNjQ4N2E4IiwidiI6ImQ0NTAxZDc5LWFiYTEtMTFlYy1hZjYzLTRhNmY0YTRkNzc3YSIsInQiOjE2NDgyNTY3MDkwNDMsImgiOiI5NTQwNzE1YTcyM2QyM2ZhZTY2MWZiODBlZmNjZGQ4ZGQ4YWMwMzE5ZGI3ZDk2MjYyOGY4NzFlNzU5NzBmZjU2In0=; ki_t=1648147211089;1648225991889;1648256407712;2;59; goodrx-v2=7dc35582596d94900a64d0061771571da77c4fd0DhqygNG8Vm/FqMuMz6Zzs4D+KiIqNdkEJuwXq9R0V3CUHBuIJMSsUw6g/6LH0boPXF7rWBTF2AAvtMAMz2bOJWUuL9ONDEjoI4X/w1PauTnWt392tWPxq55zQ8NpzQwHGQ55VaJLy2v3j4L4bua1R+6ihjhV8oAh529Bkcvs9t69fiXW4mO5roFt92WBe4amulxXIIPsrGTnXxPORcc3qINEvX0J6yuqEGmnzbDaZxbzHLf/7eI6EicfnNASiLwGSD8rR3T/pFIN2zbtuoZZSFUHDq1v/PsxXVSZOrjCEE7ni51JEupuuOTNyCjRX1Ilsxd+Dua1Tu3Lm9EfsbBnTS/UAgBbl8dvq0C/sENuyDTggtCpFJ3/heGke8R9H42twcRmjKzzMvM8UU6lDTQhm1+KiPn85MLSF8+oJAGHENR+IiezlJlvfJg8U3hygy99Z0vMGVtRTzWr/x/1E2cub1TYoRGYx6DVlI2LAmJzd1CVj/bJgP4WNWdv1Q9CIc9gwkX+PMQfpaLAcTSjgHf0T8QVpnAwtwVzrsZbRSTeBrSssSIIcfRr2K6rbhxwNoXUDfI9pT0CBHX976C1MQA/aG3exvcItZI=',
'referer': 'https://www.goodrx.com/',
'sec-ch-ua': '"Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': "macOS",
'sec-fetch-dest': 'script',
'sec-fetch-mode': 'no-cors',
'sec-fetch-site': 'same-origin',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36'
}

In [4]:
pages_letter = list(string.ascii_lowercase)
#pages_letter = ['y','z']

In [5]:
main_url = 'https://www.goodrx.com/drugs/'

In [18]:
top_rx = []
for page in tqdm(pages_letter):
    time.sleep(0.5)
    res = req.get(main_url + page, headers = headers_s, allow_redirects=True)
    #res.status_code
    soup = bs(res.content)
    top_rx_f = soup.find_all('span', {'class' : 'drugDisplay-1gY8X'})
    for x in top_rx_f:
        y = re.sub('/', "", x.get_text()) #[@,$,\D], [a-zA-Z]
        #print(y)
        i = y.lower().split()
        f = '-'.join(i)
        top_rx.append(f)
        

100%|██████████| 26/26 [00:35<00:00,  1.36s/it]


In [20]:
#top_rx.to_excel('')
#drugs = ['atorvastatin', 'amlodipine', 'armodafinil']
top_drugs_grx = pd.DataFrame(top_rx, columns=['names'])

In [21]:
top_drugs_grx.tail()

,names
489,zylet
490,zenpep
491,zorvolex
492,zortress
493,zyprexa


In [75]:
top_rxs = pd.read_excel('goodrx_med_list.xlsx',index_col = 0)

In [76]:
top_rx = top_rxs.loc[:,'names']

In [102]:
top_rx.loc[470]

'yaz'

In [103]:
main_url = 'https://www.goodrx.com/drugs/'

In [7]:
drugs_f = []
prices_re = []
prices_f = []
pharmacies_f = []
for rx in tqdm(top_rx[470:]):
    time.sleep(0.5)
    res = req.get('https://www.goodrx.com/' + rx)
    #print('\n' + rx)    
    soup = bs(res.content, 'html.parser')
    
    pharmacies = soup.find_all('span', {'class' : 'goldAddUnderline-3T_sh'})
    for pharm in pharmacies:
        pharmacies_f.append(pharm.get_text())
        drugs_f.append(rx)
    
    prices_grx = soup.find_all('div', {'class' : 'display-2N4dk'})
    for x in prices_grx:
        i = x.get_text()
        y = i.split()
        #prices_f.append(re.sub('[a-zA-Z]', "", y[-1])) #[@,$,\D]
        f = re.sub('[,$a-zA-Z]', "", y[-1])
        l = len(f)
        prices_f.append(float(f[:l-1]))

NameError: name 'top_rx' is not defined

In [105]:
print(len(drugs_f))
print(len(pharmacies_f))
print(len(prices_f))
#old was 886

126
126
126


In [94]:
#pharmacies_f.pop()

'Costco'

In [107]:
print(drugs_f)

['yaz', 'yaz', 'yaz', 'yaz', 'yaz', 'yaz', 'yaz', 'yaz', 'yaz', 'yasmin', 'yasmin', 'yasmin', 'yasmin', 'yasmin', 'yasmin', 'yasmin', 'yasmin', 'yasmin', 'yosprala', 'yosprala', 'yosprala', 'yosprala', 'yosprala', 'yosprala', 'yosprala', 'yosprala', 'yosprala', 'zolpidem', 'zolpidem', 'zolpidem', 'zolpidem', 'zolpidem', 'zolpidem', 'zolpidem', 'zolpidem', 'zolpidem', 'zolpidem-er', 'zolpidem-er', 'zolpidem-er', 'zolpidem-er', 'zolpidem-er', 'zolpidem-er', 'zolpidem-er', 'zolpidem-er', 'zolpidem-er', 'zonisamide', 'zonisamide', 'zonisamide', 'zonisamide', 'zonisamide', 'zonisamide', 'zonisamide', 'zonisamide', 'zonisamide', 'ziprasidone', 'ziprasidone', 'ziprasidone', 'ziprasidone', 'ziprasidone', 'ziprasidone', 'ziprasidone', 'ziprasidone', 'ziprasidone', 'zaleplon', 'zaleplon', 'zaleplon', 'zaleplon', 'zaleplon', 'zaleplon', 'zaleplon', 'zaleplon', 'zaleplon', 'zyrtec-d', 'zyrtec-d', 'zyrtec-d', 'zyrtec-d', 'zyrtec-d', 'zyrtec-d', 'zyrtec-d', 'zyrtec-d', 'zyrtec-d', 'zafirlukast', 'za

In [106]:
subs = {'Drug': [], 'Pharmacy':[],'Price':[]}

In [108]:
subs['Drug'] = drugs_f
subs['Pharmacy'] = pharmacies_f
subs['Price'] = prices_f

In [109]:
sub_df = pd.DataFrame(subs, columns=['Drug','Pharmacy','Price'])

In [110]:
sub_df

,Drug,Pharmacy,Price
0,yaz,CVS Pharmacy,21.95
1,yaz,Walgreens,27.56
2,yaz,Hy-Vee,16.94
3,yaz,Costco,16.89
4,yaz,Walmart,23.37
...,...,...,...
121,zyprexa,Walmart,9.00
122,zyprexa,Kroger Pharmacy,11.25
123,zyprexa,Target (CVS),80.20
124,zyprexa,Walmart Neighborhood Market,9.00


In [111]:
sub_df.to_excel("yaz_to_zyprexa.xlsx")

In [ ]:
stop

In [ ]:
for p in range(0,len(pharmacies)):
        print(prices[p])
        print(pharmacies[p].get_text())
        #print('\n')


In [ ]:
for x in drugs:
    res = req.get('https://www.goodrx.com/' + x)
    

In [8]:
soup = bs(res.content, 'html.parser')


NameError: name 'res' is not defined

In [ ]:
prices_grx = soup.find_all('div', {'class' : 'display-2N4dk'})

In [ ]:
prices_filter = []
prices = []

In [ ]:
for x in prices_grx:
    i = x.get_text()
    y = i.split()

    #print("\n" + i)
    prices_filter.append(re.sub('[a-zA-Z]', "", y[-1])) #[@,$,\D]
    

In [ ]:
for x in prices_filter:
    l = len(x)
    prices.append(x[:l-1])

In [ ]:
for x in range(0,len(prices)):
    print(prices[x])

In [ ]:
pharmacies = soup.find_all('span', {'class' : 'goldAddUnderline-3T_sh'})

In [ ]:
for x in range(0,len(pharmacies)):
    print(pharmacies[x].get_text())
    